## 1.1 Setup

### _Show that Eq. (2) is unitary._

Equation 2.2

$$
  \phi^{(n+1)} = \Bigg( \frac{1-a}{1+a} \Bigg) \phi^{(n)},
$$

where $a \equiv iH\Delta t/2$, can be derived by applying the Hamiltonian to the average of the current and next state vector, $(\phi_{i} + \phi_{i+1})/2$ (which we define as $\chi/2$ below), and making the approximaton $\partial \phi / \partial t \rightarrow \Delta \phi / \Delta t = (\phi_{i} + \phi_{i+1})/ \Delta t$. Since this updating scheme is a linear approximation in time, $A \equiv (1+a)^{-1}(1-a)$ should unitarity as $a \rightarrow 0$, which occurs as $\Delta t \rightarrow 0$. To show this, we start by observing that

$$
  (1+a)(1-a) = 1 - a^{2}
$$

giving, upon left multiplication by $(1+a)^{-1}$,

$$
  (1+a)^{-1} = (1-a) - (1+a)^{-1}a^2 \simeq (1-a)
$$

to first order. Thus

$$
  A = (1+a)^{-1}(1-a) \simeq (1-a)(1-a) = 1 - 2a + a^2.
$$

We'd like to show that $A^{T}A \rightarrow 1$ in the small $\Delta t$ limit in order to demonstrate unitarity:

$$
  A^{T}A \simeq (1 - 2a + a^2)^{T} (1 - 2a + a^2) = (1 - 2a + a^2)^{T} (1 - 2a + a^2) = (1 - 2(a^{T}+a) + O(a^2))
$$

which clearly converges to the identity matrix as $\Delta t \rightarrow 0$.

### _Determine units._

Note that the factors $m\epsilon^2/(\hbar\Delta{t})$ and $m\epsilon^2 V/\hbar^2)$ are dimensionless. $\hbar^2/m$ is implicitly set to 1 in the derivation of Eq. 2.2, so these terms reduce to $\epsilon^2/(\sqrt{m}\Delta t)$ and $\epsilon^2 V$. We would like to pick units such that the first of these terms equals unity. thus simplifying the $\chi$ equation (eq. 6) to

$$
  \chi^{(n)}_{j+1} + \Bigg( -2 + 4i - 2\epsilon^2 V_{j} \Bigg)\chi^{(n)}_{j} + \chi^{(n)}_{j-1} = 8i\phi^{(n)}_{j}.
$$

Setting $\epsilon^2/(\sqrt{m}\Delta{t}) = 1$ requires that we pick units where $m = (\epsilon^2 / \Delta{t})^2$; we are then free to pick $\epsilon$ and $\Delta t$.

We need $\epsilon$ and $\Delta t$ to be small enough for the discretized Laplacian to work well. We're working in a box of length $L = 2$ (in our length units), so $\epsilon = L/N = 2/N$ where $N+1$ is defined to be the number of discrete positions (endpoints included). In order to simulate wave numbers of order 1,000 with high accuracy, we should pick $N$ to be on the order of 100,000, which should be very manageable for a normal laptop (Since taking matrix products with a tridiagonal matrix requires only $O(N)$ calculations). Initially I set $N = 20,000$, giving $\epsilon = 10^{-4}$, but this resulted in extremely slow evolution, so I instead set $N = 2,000$ (after trial and error), giving $\epsilon = 10^{-3}$. If we'd like to set $m = \hbar^2 = 1$, we need $\Delta{t} = \epsilon^2$, in this case giving $\Delta{t} = 10^{-6}$.

The main point, though, is that we're choosing units where $\hbar = 1$, $m = 1$, and $\epsilon^2 / \Delta{t} = 1$.

## 1.2 Evolution of a wavepacket

Create a wavepacket function constructor that takes $k_{0}$ and $\sigma$ as parameters and have it return a wavefunction $\phi (x)$:

$$
  \Phi (k_{0}, \sigma, x_{0}) = \phi(x) = e^{ik_{0}x} e^{-(x-x_{0})^2/(2\sigma^2)}
$$

Using the Kronecker delta functions from `Laplacian` module made for the last problem set, construct the $\chi$ evolution matrix from Eq. 6, rewriting it as and use it (along with equation 5) to update the state vector, $\phi$. Implement both the $\chi$ evolution matrix and the $\phi$ evolution function in a new module called `Evolution`. Rewrite Eq. 6 using Einstein summation notation as

$$
   A_{i,j}\chi^{(n)}_{j} = \phi^{(n)}_{i}
$$

where

$$
  A_{i,j} \equiv -\frac{i}{8}\Big( \delta_{i+1,j} - 2 \big( 1+\epsilon^2 V_{j} -2i \big) \delta_{i,j} + \delta_{i-1,j} \Big) = -\frac{i}{8}\delta_{i+1,j} +\Big(\frac{(1 + \epsilon^2 V_{j})}{4}i + 0.5\Big)\delta_{i,j} -\frac{i}{8}\delta_{i-1,j}
$$

which can be efficiently solved by LU factorization and plugged into Eq. 5:

$$
  \phi^{(n+1)} = \chi^{(n)} - \phi^{(n)}.
$$

We can preserve the boundary conditions $\phi = 0$ by setting our initial $\phi$ to zero at those points and then setting the first and last rows and columns of A to zero, except for `A[1,1]` and `A[end,end]`, which should both equal 1 in order to satisfy the Thomas algorithm stability requirement that A be diagonally dominant.

I tried the simulation with a few combinations of wavenumber $(k)$ and wavepacket width $(\sigma)$. The simulation function runs for 16,000 steps and samples a frame every 50 steps for the animations shown below. Other parameters are as noted above. The simulation is implemented in a way that makes it quite straightforward to run simulations with different combinations of parameters.

### Wave Packet propagating in Free Space with k = 100, $\sigma$ = 0.02
![k=100, sigma=0.02](M_re_100_002.gif)

### Wave Packet propagating in Free Space with k = 100, $\sigma$ = 0.05
![k=100, sigma=0.05](M_re_100_005.gif)

### Wave Packet propagating in Free Space with k = 200, $\sigma$ = 0.02
![k=200, sigma=0.02](M_re_200_002.gif)

### Wave Packet propagating in Free Space with k = 200, $\sigma$ = 0.05
![k=200, sigma=0.05](M_re_200_005.gif)

The wave packet with $k = 200$, $\sigma = 0.05$ showed little dispersion as it evolved, so I picked these parameters for part 3. Run [plot2.m](plot2.m) to see better versions of the graphs in MATLAB. You can restart the GIFs by refreshing the page.

I also tested the stability of the algorithm by running for 2,000 steps before taking the complex conjugate of $\phi$ (thus reversing its momentum) and simulating for another 2,000 steps to return it to its original state.

### Wave Packet propagating for 2,000 Steps before being reversed, with k = 200, $\sigma$ = 0.05
![k=200, sigma=0.05](M_rev_re_200_005.gif)

The average absolute error between initial and final states was `-3.7646e-17`, which is plenty stable for our purposes. Taking `mean(abs(y2(2:end-1)).^2) - mean(abs(y1(2:end-1)).^2)` (where y2 and y1 are the final and initial states, respectively) returned precisely zero, meaning that the area of the graphs is preserved to an even higher degree of accuracy. Since inverting the time step creates the inverse matrix, we can conclude that our time evolution matrix is unitary.

## 1.3 Scattering from a Step Function Potential

I tried a few different values for the potential energy step. For these, I started the wave packet at $x=-0.5$ so that the right hand side of the field would be approximately zero initially. I set up my simulation to stop running as soon as the magnitude of the wave near the right edge became greater than $10^{-3}$, at which point it would record the average value of the square of the wavefunction on both left and right hand sides of the step, and then normalize both of those averages to get the transmission and reflection probabilities. There is a very steep drop-off in transmission probability between $V = 10,000$ and $V = 30,000$.

![Transmission Probability vs. Potential](VTR.svg)

Note the change in behavior:

### $V = 10,000$
![Wave packet hitting step](M_step_re_200_10000.gif)
### $V = 20,000$
![Wave packet hitting step](M_step_re_200_20000.gif)
### $V = 30,000$
![Wave packet hitting step](M_step_re_200_30000.gif)

Clearly the large V limit of total reflection/zero transmission is approached.

## 1.4 Decay of an Unstable State

Setting $w=0.2$ and using our Hamiltonian on the suggested starting function, `(HH * sin(π*[-1:1e-3:1] ./ 0.2)) ./ sin(π*[-1:1e-3:1] ./ 0.2)`, gives an energy of 123.368. Set b to 50 and h to 125, so that it is classically impossible to escape from the pocket but so that tunneling is minimally suppressed. Then vary s (while keeping the starting wavefunction fixed) and use the previous method to detect when the wavefunction has "leaked" to the boundaries; the more of the wave that's leaked, the closer we are to resonance.

![Leakage probability vs pocket width, close to start](decay_close.svg)
![Leakage probability vs pocket width](decay.svg)

Unfortunately, I was not able to find a resonance state near the recommended starting point; it was only by drastically decreasing the width of the pocket that leakage was increased, but this is due to the fact that, by cutting into the sine function on either side, we're introducing higher energy harmonics that aren't classically confined to the pocket (put another way, we're decreasing positional uncertainty and hence increasing energy uncertainty). Given more time, I would have tried varying the heights of the potential barriers.

## 1.5 Scattering

We can reuse our approach from part 3 to scatter off of the potential from part 4. Since I couldn't find the resonant state in part 4, I wasn't able to determine its energy, so I'm just scattering wave packets that are close to what we started off with.

![Transmission vs. momentum](scatter.svg)

This was with the wave packet from earlier; clearly its momentum is too high. Unfortunately, I ran out of time on this one and didn't scale down the energy of the wave packet to that of the confined wave from part 4. Had I done so, my approach would have been the same as with the earlier problems (i.e. varying the parameters, in this case energy of the wave packet, and writing automated routines to detect desired behaviour, in this case, a high transmission coefficient).